# Title: Exploring the Relationship Between Disney Movie Profits and Director Selection
## Author Name: Yeonok Yi

### **Introduction**

**Question(s) of interests**

This analysis aims to explore the connection between Disney movie profits and the directors involved. Notably, certain directors appear to be repeatedly hired by Disney. This investigation seeks to understand if a director's track record of delivering high box office returns influences their rehiring by Disney for future projects. I would expect the director who made more gross profit to get hired more than others.

**Dataset description**

I was given 5 data sets, but for this analysis, emphasis is placed solely on the information about directors and gross profits. The selected datasets, namely 'disney_movies_total_gross.csv' and 'disney-director.csv,' are essential for the analysis. Their descriptions are outlined below:
- disney_movies_total_gross.csv
    - This dataset encompasses fundamental movie details such as titles, release dates, genres, gross profits, and more. For this analysis, only the movie titles and corresponding gross profits will be used.
- disney-director.csv
    - A simple dataset with movie titles and their directors. This dataset is essential to connect between directors and gross profit which I want to check.

### **Methods & Results**

As the primary focus lies in establishing a correlation between directors and the gross profits generated from movies, the analysis will rely on tables containing information specifically about gross profits and directors.

To commence the analysis, the initial step involves the importation of these essential libraries and tables followed by conducting basic visualizations for further insights.

In [1]:
# Import libraries
import altair as alt
import pandas as pd

In [2]:
# import all the required files
gross = pd.read_csv("data/disney_movies_total_gross.csv")
director = pd.read_csv("data/disney-director.csv")

Lets see what the tables look like.

In [3]:
gross.head()

movie_title  release_date      genre MPAA_rating  \
0  Snow White and the Seven Dwarfs  Dec 21, 1937    Musical           G   
1                        Pinocchio   Feb 9, 1940  Adventure           G   
2                         Fantasia  Nov 13, 1940    Musical           G   
3                Song of the South  Nov 12, 1946  Adventure           G   
4                       Cinderella  Feb 15, 1950      Drama           G   

    total_gross inflation_adjusted_gross  
0  $184,925,485           $5,228,953,251  
1   $84,300,000           $2,188,229,052  
2   $83,320,000           $2,187,090,808  
3   $65,000,000           $1,078,510,579  
4   $85,000,000             $920,608,730

In [4]:
director.head()

name        director
0  Snow White and the Seven Dwarfs      David Hand
1                        Pinocchio  Ben Sharpsteen
2                         Fantasia    full credits
3                            Dumbo  Ben Sharpsteen
4                            Bambi      David Hand

This time, we will get some other information about the sets table.

In [5]:
gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   movie_title               579 non-null    object
 1   release_date              579 non-null    object
 2   genre                     562 non-null    object
 3   MPAA_rating               523 non-null    object
 4   total_gross               579 non-null    object
 5   inflation_adjusted_gross  579 non-null    object
dtypes: object(6)
memory usage: 27.3+ KB


The sets table has  579 rows and  6 columns. **Every movie_title has a total_gross**. Genre and MPAA_rating have some null data, but it won't affect my analysis today.

In [6]:
director.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      56 non-null     object
 1   director  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


The sets table has  56 rows and  2 columns. There is **no null data** on namd and director


I just need the title and gross from the gross table. To do this, I will use **loc** to make it easy to use to find the connection with director information.

In [7]:
# group by year and compute the average number of parts.
set_gross=gross.loc[:,["movie_title", "total_gross",]]

# check the table to see if I made it righ.
set_gross.head()

movie_title   total_gross
0  Snow White and the Seven Dwarfs  $184,925,485
1                        Pinocchio   $84,300,000
2                         Fantasia   $83,320,000
3                Song of the South   $65,000,000
4                       Cinderella   $85,000,000

Since we removed all the unrelated data, time to **merge** two tables.

In [8]:
# Merge two tables using an inner join to focus on the relationship between directors and gross profits.
# Eliminate duplicated information by dropping the 'name' column from the director table before merging the datasets.

profitable_director = (set_gross.merge(director, 
                                  left_on='movie_title', 
                                  right_on='name', 
                                  how='inner', 
                                  indicator=True).drop(columns=['name', '_merge']))
profitable_director.head()

movie_title   total_gross         director
0  Snow White and the Seven Dwarfs  $184,925,485       David Hand
1                        Pinocchio   $84,300,000   Ben Sharpsteen
2                         Fantasia   $83,320,000     full credits
3                       Cinderella   $85,000,000  Wilfred Jackson
4                       Cinderella  $201,151,353  Wilfred Jackson

Since the total_gross column's D-type is object. I'll need to change it to **float**

In [9]:
# Change the D-type with astype function
profitable_director['total_gross'] = profitable_director['total_gross'].replace('[$,]', '', regex=True).astype(float)

#Let's check if I did right.
profitable_director.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_title  49 non-null     object 
 1   total_gross  49 non-null     float64
 2   director     49 non-null     object 
dtypes: float64(1), object(2)
memory usage: 1.5+ KB


Since the total_gross is float, I can use it as a number now.

As an initial step in visualization, let's examine the **number of movies each director has made**.
To achieve this, I will employ the groupby function to group the data by director and then use the count function to determine the quantity of movies attributed to each director.

In [10]:
# Count and sort in descending order to see if there are a lot of directors who made movies more than once.
count_movie = profitable_director.groupby(['director'])['movie_title'].count().sort_values(ascending=False)

# Reset the index so we can plot using altair
count_movie = count_movie.reset_index()
count_movie.head(10)

director  movie_title
0  Wolfgang Reitherman            9
1         Ron Clements            7
2       Gary Trousdale            3
3          Mark Dindal            2
4           Chris Buck            2
5      Wilfred Jackson            2
6       Clyde Geronimi            2
7  Stephen J. Anderson            2
8         Mike Gabriel            2
9         Ralph Zondag            1

I confirmed Disney has hired some directors over and over again with this table. Maximum 9 times to minimum 2 times.

Now that we can generate a **bar plot** to visualize it.

In [11]:
count_bar = alt.Chart(profitable_director, width=500, height=300).mark_bar(color='green', opacity=0.5).encode(
            x='director',
            y=alt.Y("count()", title="Numbers of Movies")).properties(title='Number of movies each director has made')

count_bar

alt.Chart(...)

In the dataset, some directors have only worked on a single movie. My aim is to compare their profitability with directors who have worked on multiple films. To achieve this, I'll get **the average gross profit from a group of single-movie directors while I calculate each individual average gross profit for directors involved in multiple productions**. I believe this way we can find the connection between the size of numbers and gross profits rather than compare only two big groups such as single-movie directors and multi-movie directors.

For this comparative analysis, a **custom function** in a script will be utilized. It will sort the data based on directors' movie counts and compute individual director profitability for in-depth comparison.

In [12]:
# import the custom script
import script as ps

# run it on the data
ave_director = ps.sort_by_movies(profitable_director)
ave_director

(143959841.16666666,
 {'Wolfgang Reitherman': 107334398.0,
  'Ron Clements': 120030687.85714285,
  'Gary Trousdale': 134381079.33333334,
  'Chris Buck': 285914914.0,
  'Mark Dindal': 112341619.0,
  'Stephen J. Anderson': 62257508.5,
  'Wilfred Jackson': 143075676.5,
  'Mike Gabriel': 84755617.0,
  'Clyde Geronimi': 171827859.0})


I obtained the calculation result and now need to **convert it into a DataFrame** to visualization

In [13]:
# Create a DataFrame by transposing the dictionary and renaming the columns
df_ave = pd.DataFrame([ave_director[1]]).T.reset_index()
df_ave.columns = ['Director', 'Values']

# Rename the first column to 'Single_Movie' and the first value of the tuple to match the column
df_ave.columns = ['Name', 'Average Gross Profit']

# Append the first element of the tuple as a new row to the DataFrame and name it as "Single Movie Maker"
df_ave.loc[len(df_ave)] = ['Single Movie Maker', ave_director[0]]
df_ave

Name  Average Gross Profit
0  Wolfgang Reitherman          1.073344e+08
1         Ron Clements          1.200307e+08
2       Gary Trousdale          1.343811e+08
3           Chris Buck          2.859149e+08
4          Mark Dindal          1.123416e+08
5  Stephen J. Anderson          6.225751e+07
6      Wilfred Jackson          1.430757e+08
7         Mike Gabriel          8.475562e+07
8       Clyde Geronimi          1.718279e+08
9   Single Movie Maker          1.439598e+08

Look like I have profer DataFrame but I missed how many movies they made.
I'll add the information.

In [14]:
# Merged count_movie DataFrame containing the number of movies counted for each director.
director_info = (df_ave.merge(count_movie, 
                                  left_on='Name', 
                                  right_on='director', 
                                  how='left', 
                                  indicator=True).drop(columns=['director', '_merge']))
director_info

Name  Average Gross Profit  movie_title
0  Wolfgang Reitherman          1.073344e+08          9.0
1         Ron Clements          1.200307e+08          7.0
2       Gary Trousdale          1.343811e+08          3.0
3           Chris Buck          2.859149e+08          2.0
4          Mark Dindal          1.123416e+08          2.0
5  Stephen J. Anderson          6.225751e+07          2.0
6      Wilfred Jackson          1.430757e+08          2.0
7         Mike Gabriel          8.475562e+07          2.0
8       Clyde Geronimi          1.718279e+08          2.0
9   Single Movie Maker          1.439598e+08          NaN

I know single movie maker is a group of directors who only made one movie for Disney. So I'll To replace the NaN value with 1.

In [15]:
director_info['movie_title'] = director_info['movie_title'].fillna(1)
director_info

Name  Average Gross Profit  movie_title
0  Wolfgang Reitherman          1.073344e+08          9.0
1         Ron Clements          1.200307e+08          7.0
2       Gary Trousdale          1.343811e+08          3.0
3           Chris Buck          2.859149e+08          2.0
4          Mark Dindal          1.123416e+08          2.0
5  Stephen J. Anderson          6.225751e+07          2.0
6      Wilfred Jackson          1.430757e+08          2.0
7         Mike Gabriel          8.475562e+07          2.0
8       Clyde Geronimi          1.718279e+08          2.0
9   Single Movie Maker          1.439598e+08          1.0

To ascertain a potential proportional relationship between the number of movies and the gross profit, I'll create two bar plots. One will represent the 'Number of Movies by Director,' acting as the **standard of comparison**, while the other will showcase 'Average Gross Profit by Director.' If my hypothesis holds true, these two graphs should exhibit similar shapes.

To ensure a fair comparison between the two graphs, I'll sort the data in **ascending order based on the number of movies** per director.

In [16]:
# Create the bar plot for the comparison
Comparison_bar = alt.Chart(director_info, width=500, height=300).mark_bar(color='blue', opacity=0.5).encode(
    x=alt.X('Name', sort=alt.EncodingSortField(field='movie_title', order='ascending'), title="Director"),
    y=alt.Y('movie_title:Q', title="How many movies made")
).properties(
    title='Number of Movies by Director'
)

Comparison_bar

alt.Chart(...)

In [17]:
# Creat the bar plot for the relationship with Gross Profit
Relationship_bar = alt.Chart(director_info, width=500, height=300).mark_bar(color='pink', opacity=0.8).encode(
            x = alt.X('Name', sort=alt.EncodingSortField(field='movie_title', order='ascending'), title="Director"),
            y='Average Gross Profit').properties(title='Average Gross Profit by Director')

Relationship_bar

alt.Chart(...)

It appears that the two graphs **do not exhibit a similar shape**.
A notable example is the gross profit of the 'Single Movie Maker' group, comprising directors who only made one movie, which surpasses that of Wolfgang Reitherman, despite Reitherman having directed the most movies for Disney.

### Discussions
In this analysis of the Disney dataset, I sought to unveil underlying connections based on my initial observations. Before delving into answering questions, I conducted essential checks, discovering a director who had worked on Disney movies nine times, contrasting with the majority of directors who worked on only a single Disney movie. This revelation sparked the analysis, hinting at potentially significant reasons behind the scenes.

The absence of a noticeable correlation between a director's movie count for Disney and the resulting gross profit was surprising. My initial assumptions regarding this relationship might have been too pragmatic or industry-driven. The director with the highest movie count didn't make more profit compared to those who directed only a single Disney movie. This raises the possibility of other factors influencing artistic outcomes.

One aspect that leaves me unsatisfied with my analysis is the inability to create a bar plot with two different Y-axis values or merging a bar plot with a scatter plot into a single graph. This approach could have provided a clearer visual representation to show the absence of a correlation between both graphs. I believe gaining proficiency in this area will enhance not only my understanding but also that of the audience. I am eager to learn more in the upcoming classes to improve my analytical and visualization skills.

### Reference
Not all the work in this notebook is original.